In [1]:

# Thêm thư viện
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from imutils import paths
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.applications import VGG16
from keras.layers import Input
from keras.models import Model
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Flatten
import numpy as np
import random
import os

Using TensorFlow backend.


In [2]:
# Lấy các đường dẫn đến ảnh.
image_path = list(paths.list_images('dataset/'))

# Đổi vị trí ngẫu nhiên các đường dẫn ảnh
random.shuffle(image_path)

In [3]:
# Đường dẫn ảnh sẽ là dataset/tên_phương_tiện/tên_ảnh ví dụ dataset/Bluebell/image_0241.jpg nên p.split(os.path.sep)[-2] sẽ lấy ra được tên phương tiện
labels = [p.split(os.path.sep)[-2] for p in image_path]

# Chuyển tên các loài hoa thành số
le = LabelEncoder()
labels = le.fit_transform(labels)

# One-hot encoding
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [4]:
# Load ảnh và resize về đúng kích thước mà VGG 16 cần là (224,224)
list_image = []
for (j, imagePath) in enumerate(image_path):
    # Đọc ảnh dưới dạng PIL
    image = load_img(imagePath, target_size=(224, 224))
    # Chuyển ảnh về dạng Numpy
    image = img_to_array(image)
    # Thêm 1 chiều nữa cho ảnh (1,224,224,3)
    # image = np.expand_dims(image, 0)
    # Lấy giá trị điểm ảnh trừ đi giá trị trung bình của RGB
    image = preprocess_input(image)   
    list_image.append(image)
    
list_image = np.array(list_image)

In [5]:
# Load model VGG 16 của ImageNet dataset, include_top=False để bỏ phần Fully connected layer ở cuối.
baseModel = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Xây thêm các layer
# Lấy output của ConvNet trong VGG16
fcHead = baseModel.output

# Flatten trước khi dùng FCs
fcHead = Flatten(name='flatten')(fcHead)

# Thêm FC
fcHead = Dense(256, activation='relu')(fcHead)

# Droput để tránh overfitting , regularization tăng hàm phạt
fcHead = Dropout(0.5)(fcHead)

# Output layer với softmax activation
fcHead = Dense(5, activation='softmax')(fcHead)

# Xây dựng model bằng việc nối ConvNet của VGG16 và fcHead
model = Model(inputs=baseModel.input, outputs=fcHead)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
# Chia traing set, test set tỉ lệ 80-20
X_train, X_test, y_train, y_test = train_test_split(list_image, labels, test_size=0.2, random_state=42)

In [7]:
# augmentation cho training data
# Thêm dữ liệu cho data
aug_train = ImageDataGenerator(rescale=1./255, rotation_range=30, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
# augementation cho test
aug_test= ImageDataGenerator(rescale=1./255)

In [8]:
# Đóng băng toàn bộ VGG
for layer in baseModel.layers:
    layer.trainable = False
    
opt = RMSprop(0.001)
model.compile(opt, 'categorical_crossentropy', ['accuracy'])
num_Epoch = 2
H = model.fit_generator(aug_train.flow(X_train, y_train, batch_size=10), 
                        steps_per_epoch=len(X_train)//10,
                        validation_data=(aug_test.flow(X_test, y_test, batch_size=10)),
                        validation_steps=len(X_test)//10,
                        epochs=num_Epoch)

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
 41/148 [=======>......................] - ETA: 4:35 - loss: 7.3843 - acc: 0.3488

KeyboardInterrupt: 

In [ ]:
# Mở đóng băng toàn bộ và train cùng các layer được thêm
for layer in baseModel.layers:
    layer.trainable = True
    
opt = RMSprop(0.001)
model.compile(opt, 'categorical_crossentropy', ['accuracy'])
num_Epoch = 2
# steps_per_epoch : Số lần tính gradient trong 1 epoch
# valadation_step: Tương tự nhưng trên tập validation
# Đối với fit_generator cho phép bỏ qua những dữ liệu trùng nhau và có nhiều tuỳ biến
H = model.fit_generator(aug_train.flow(X_train, y_train, batch_size=10), 
                        steps_per_epoch=len(X_train)//10,
                        validation_data=(aug_test.flow(X_test, y_test, batch_size=10)),
                        validation_steps=len(X_test)//10,
                        epochs=num_Epoch)

In [ ]:
model.summary()

In [11]:
preds = model.evaluate(X_test, y_test)
print ("Loss = " + str(preds[0]))
print ("Accuracy of VGG16 for Test set  = " + str(preds[1]))

372/372 [==============================] - 132s 355ms/step
Loss = 12.478525448870915
Accuracy of VGG16 for Test set  = 0.22580645177313077
